# Access large areas

In the previous notebook `0_access_worldcover_products`, ESA WorldCover 2020 data was loaded and analysed over a small area of interest.

In this notebook, WorldCover data in the spatial extend of an entire country will be searched, downloaded and analysed.

The entire country of Belgium will be analysed. As the country area interestects with multiple WorldCover rasters, the `rio-tiler` package will be used and is therefore required to run this notebook.

## Find the STAC products for the area of interest

In [1]:
# Load the area of interest geometry (belgium)
import geopandas as gpd

# Load countries border dataset
countries_dataset = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

belgium_entry = countries_dataset.loc[countries_dataset['name'] == 'Belgium']
belgium_entry

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
129,11491346,Europe,Belgium,BEL,508600.0,"POLYGON ((6.15666 50.80372, 6.04307 50.12805, ..."


In [2]:
bbox = list(belgium_entry.geometry.bounds.values.squeeze())
bbox

[2.5135730322461427, 49.529483547557504, 6.15665815595878, 51.47502370869813]

In [3]:
from rio_tiler.io import STACReader
from rio_tiler.profiles import img_profiles
from rio_tiler.models import ImageData
import pystac_client
import geopandas as gpd

# Log in to the terrascope STAC API
TERRASCOPE_STAC_API = 'https://services.terrascope.be/stac/'
COLLECTION_ID = 'urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1'

client = pystac_client.Client.open(TERRASCOPE_STAC_API)
search = client.search(
    max_items=100,
    collections=[COLLECTION_ID],
    bbox = bbox
)

result_items = search.get_all_items().items

result_items

[<Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E000>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E003>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N48E006>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E000>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E003>,
 <Item id=urn:eop:VITO:ESA_WorldCover_10m_2020_AWS_V1:ESA_WorldCover_10m_2020_v100_N51E006>]

In [4]:
# We got 3 result items, let's get the access links to the rasters
result_items_assets = list(map(lambda item: item.assets.get('ESA_WORLDCOVER_10M_MAP').href, result_items))
result_items_assets

['s3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E000_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E003_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N48E006_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E000_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E003_Map.tif',
 's3://esa-worldcover/v100/2020/map/ESA_WorldCover_10m_2020_v100_N51E006_Map.tif']

## Parallel loading of tiles with rio-tiler

Using the `rio_tiler.mosaic.mosaic_reader` method, we read the multiple assets found in our STAC catalog to create a mosaic

from rio_tiler.mosaic import mosaic_reader
from rio_tiler.io import STACReader

import boto3
from botocore import UNSIGNED
from botocore.client import Config

import os


# As we do not need to identify, we set the 

client = boto3.client('s3', config=Config(signature_version=UNSIGNED))# Define the function to access tile data
def tiler(asset, *args, **kwargs):
    with STACReader(asset, fetch_options={"request_pays": False,
                                          "client": client}) as src:
        return src.tile(*args, **kwargs)

image, used_assets = mosaic_reader(
    result_items_assets,
    tiler,
    threads=2
)

print(used_assets)
print(image)

## Parallel access of tiles trough rasterio

Using `rasterio`, we will load all the tiles in multiple threads

In [9]:
import rasterio
import os
import threading

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

# Initialize result array
results = [None] * len(result_items_assets)

# Function to load the dataset from a remote link
def load_dataset(index, path):
    with rasterio.open(path, 'r') as src:
        # Window on our interest area, we don't want to download all the entire tiles.
        interest_window = rasterio.windows.from_bounds(*bbox, transform=src.transform)

        data = src.read(1, 
                        out_dtype=rasterio.uint8,
                        window=interest_window)
        crs = src.crs
        transform = src.transform
    results[index] = {'data': data, 'crs': crs, 'transform': transform}

# Initialzes all the threads with the asset link parameters 
thread_pool = [threading.Thread(target=load_dataset, args=(index, asset,)) 
               for index, asset in enumerate(result_items_assets)]

# Starts all the threads
for thread in thread_pool:
    thread.start()
    
# Wait that all the threads have finished their job
for thread in thread_pool:
    thread.join()
    
results

[{'data': array([[40, 40, 40, ..., 30, 30, 40],
         [40, 40, 40, ..., 30, 30, 30],
         [40, 40, 40, ..., 30, 30, 30],
         ...,
         [40, 40, 40, ..., 10, 10, 10],
         [40, 40, 40, ..., 10, 10, 10],
         [40, 40, 40, ..., 10, 10, 10]], dtype=uint8),
  'crs': CRS.from_epsg(4326),
  'transform': Affine(8.333333333333333e-05, 0.0, 0.0,
         0.0, -8.333333333333333e-05, 51.0)},
 {'data': array([[40, 40, 40, ..., 30, 30, 30],
         [30, 30, 30, ..., 30, 30, 30],
         [30, 30, 30, ..., 30, 30, 30],
         ...,
         [10, 10, 10, ..., 40, 30, 30],
         [10, 10, 10, ..., 30, 30, 30],
         [10, 10, 10, ..., 30, 30, 30]], dtype=uint8),
  'crs': CRS.from_epsg(4326),
  'transform': Affine(8.333333333333333e-05, 0.0, 3.0,
         0.0, -8.333333333333333e-05, 51.0)},
 {'data': array([[30, 30, 10, ..., 40, 40, 40],
         [30, 30, 10, ..., 40, 40, 40],
         [30, 30, 10, ..., 40, 40, 40],
         ...,
         [30, 30, 30, ..., 30, 10, 10],
  

# Recreate the full image from the extracted data

In [11]:
for res in results:
    print(res['data'].shape)

(17646, 5837)
(17646, 36000)
(17646, 1879)
(5700, 5837)
(5700, 36000)
(5700, 1879)


In [ ]:
import tempdir

tempfile = NamedTemporaryFile()

with rasterio.open()